# Ejercicio 1.2
## Jorge Pablo Ávila Gómez 

## Ejercicio 1.3: Mejorando el país con mejores clientes.
Partiendo del código implementado para el ejercicio anterior, mejóralo para que,
en el caso de que haya más de un país empatado con el mayor número de
buenos clientes, se devuelvan todos esos países. Utilizando los ficheros de datos
sobre países y clientes vistos anteriormente, la salida de este programa
MapReduce debería ser la que se muestra en la Figura 3. Hay
que tener en
cuenta que la entrada del reducer es una clave y un generator que contiene los
valores que comparten la misma clave. Este ejercicio tiene una puntuación de
hasta 4 puntos sobre 10.

### Diseño del programa MapReduce
#### Limpieza de los datos
En la primera parte del ejercicio se realiza la misma limpieza de datos realizada en el Ejercicio 1.1

Se elimina la cabezera del documento countries y se limpian las columnas quitando la coma y el entrecomillado:

`"Bolivia, Plurinational State of" -> Plurinational State of Bolivia`
#### Diseño MapReduce
El ejercicio se resuelto usando dos pasos, en el primero se ejecuta un mapper y un reducer, y en el segundo paso un reducer.
- Map. (`mapper_find_bueno`) El código de este mapper es idéntico al del ejercicio 1.1 y 1.2. El map se usa para leer los dos documentos y extraer la información necesaria.
Se necesita extraer la columna nombre del pais del documentro countries y la columna valoración == 'bueno' del documento clients. Las dos tablas se encuentran enlazadas por la columna country2digit.

Para el documento countries se separa en dos columnas, nombre del pais y country2digit. Se envia como clave la columna country2digit y como valor una lista formada por la letra 'A' y el nombre del pais.

Para el documento clients se separa en tres columnas, nombre del cliente, valoración y country2digit. Aquí es importante que filtremos las columnas en las que la valoración es 'bueno'. Enviamos como clave country2digit, que es la columna que nos sirve para relacionar las dos tablas, y como valor enviamos una lista formada por la letra 'B' y como segundo elemento el número 1. Indicando que para ese pais hemos encontrado al menos 1 valoración buena.

Se utiliza las etiquetas 'A' y 'B' junto con la opción: 
```python 
SORT_VALUES = True```
Para que al reducer los valores llegen ordenados. Como sabemos, habrá un reducer para cada country2digit, es decir para cada país. Y los valores vendrán ordenados primero con la etiqueta 'A', solo habrá una y corresponde al documento countries con el nombre completo del pais, y luego todos los valores con la etiqueta 'B', procedentes del documento clients indicando que se ha encontrado una valoración buena.

- Reduce. (`reducer_count_bueno`) La función reducer va a contar el numero de valoraciones positivas que le llega de cada país. El código es idéntico al del ejercicio 1.2.

Para cada country2digit primero recibirá una lista con la etiqueda 'A' y el nombre completo del país. Luego recibirá un número de valores con la etiqueta 'B'. El reducer va sumando cuantas 'B' le llegan correspodiéndose cada una con una valoración buena.

El reducer devuelve siempre la clave `None`, para que todos los valores se dirijan al mismo reducer final. Como valores devuelve una lista, el primer elemento es el número de valoraciones positivas y el segundo elemento el nombre del país.

- Reduce. (`reducer_sort_countries`) Este reducer se encarga de devolver todos los países con el mayor número de valoraciones buenas.

A este reducer le llegan todos los valores enviados por el reducer anterior (valores con clave `None`) que además están ordenados (de menor a mayor) por la línea SORT_VALUES = True.

Como primer paso los valores se pasan a una lista y se invierte la lista, así los tenemos ordenados de mayor a menor.

Para cada elemento de la lista devolvemos el primero (el pais con más valoraciones buenas) y los siguientes elementos que tengan el mismo número de valoraciones. 
Cuando nos encontramos con un elemento con un número de valoraciones menor terminamos la función.

In [1]:
# Se crea la carpeta ejercicio103 donde se van a almacenar los documentos que se generen
! mkdir -p tp1/ejercicio103
import os
os.chdir("/media/notebooks/tp1/ejercicio103")
! pwd

/media/notebooks/tp1/ejercicio103


In [2]:
# Los ficheros countries.csv y clients.csv deben estar descargados en la carpeta /media/notebooks/tp1/ejercicio103
! cp /media/notebooks/countries.csv /media/notebooks/tp1/ejercicio103/
! cp /media/notebooks/clients.csv /media/notebooks/tp1/ejercicio103/

In [3]:
## Limpieza de countries.csv

with open("countries.csv",'r') as f, open("countries_cleaned.csv",'w') as f1:
    next(f) # salta el encabezado
    for line in f:
        if '''"''' in line: # Los paises con el problema de la coma estan entrecomillados
            loc = line.find(',') # Elimina la primera coma para no confundir con la separacion entre columnas
            new_line = line[loc+2:-5] + ' ' + line[1:loc] + line[-4:] # Reordena el nombre del pais
            f1.write(new_line)
            print(line)
            print(new_line)
        else:
            f1.write(line)

"Bolivia, Plurinational State of",BO

Plurinational State of Bolivia,BO

"Bonaire, Sint Eustatius and Saba",BQ

Sint Eustatius and Saba Bonaire,BQ

"Congo, the Democratic Republic of the",CD

the Democratic Republic of the Congo,CD

"Iran, Islamic Republic of",IR

Islamic Republic of Iran,IR

"Korea, Democratic People's Republic of",KP

Democratic People's Republic of Korea,KP

"Korea, Republic of",KR

Republic of Korea,KR

"Macedonia, the Former Yugoslav Republic of",MK

the Former Yugoslav Republic of Macedonia,MK

"Micronesia, Federated States of",FM

Federated States of Micronesia,FM

"Moldova, Republic of",MD

Republic of Moldova,MD

"Palestine, State of",PS

State of Palestine,PS

"Saint Helena, Ascension and Tristan da Cunha",SH

Ascension and Tristan da Cunha Saint Helena,SH

"Taiwan, Province of China",TW

Province of China Taiwan,TW

"Tanzania, United Republic of",TZ

United Republic of Tanzania,TZ

"Venezuela, Bolivarian Republic of",VE

Bolivarian Republic of Venezuela,VE



In [4]:
%%writefile mrjob-ejercicio103.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class MRJoin(MRJob):

    SORT_VALUES = True
    
    def steps(self):
        return [MRStep(mapper=self.mapper_find_bueno,                
                       reducer=self.reducer_count_bueno),
                MRStep(reducer=self.reducer_sort_countries),
               ]
    
    def mapper_find_bueno(self, _, line):
        splits = line.rstrip("\n").split(",")
        if len(splits) == 2: # datos de paises
            symbol = 'A'     # diferencia datos de paises de clientes
            country2digit = splits[1]
            yield country2digit, [symbol, splits[0]] #valor es el simbolo A y el nombre del pais completo
        else: #  datos de personas
            symbol = 'B' 
            country2digit = splits[2]
            if splits[1] == 'bueno': # seleccionamos solo los clientes con una valoracion buena
                yield country2digit, [symbol, 1] # enviamos solo un valor cuando hemos encontrado una val buena 
    
    def reducer_count_bueno(self, key, values):
        for value in values:
            if value[0] == 'A': # dato de pais
                country = value[1] # nombre del pais
                counter = 0
            if value[0] == 'B': # dato de persona
                counter +=1 # sumamos 1 por cada valoración buena
        else:
            if counter > 0: # solo enviamos resultados hay al menos 1 valoración buena
                yield None, [counter, country] # clave None para que llegue al mismo reducer final     

    def reducer_sort_countries(self, key, values):      
        countries = reversed(list(values)) # invierte los valores para que esten ordenados de mayor a menor
        counter = -1
        for country in countries:
            if country[0] >= counter: # el primer elemento de la lista y todos los que tienen el mismo valor
                yield country
                counter = country[0] 
            else: # cuando encontramos un elemento que tiene un valor menor no necesitamos seguir ejecutando la f
                break    
                
if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio103.py


#### Primero ejecutamos el código en local:

In [5]:
! python mrjob-ejercicio103.py /media/notebooks/tp1/ejercicio103/countries_cleaned.csv  \
/media/notebooks/tp1/ejercicio103/clients.csv > ouputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio103.root.20201030.165700.322089
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/mrjob-ejercicio103.root.20201030.165700.322089/output
Streaming final output from /tmp/mrjob-ejercicio103.root.20201030.165700.322089/output...
Removing temp directory /tmp/mrjob-ejercicio103.root.20201030.165700.322089...


In [6]:
! tail ouputlocal

3	"Spain"
3	"Guam"


#### Ejecutamos el código en hadoop:

In [7]:
! hdfs dfs -mkdir /tmp/mrjoin/ejercicio103
! hdfs dfs -put /media/notebooks/tp1/ejercicio103/countries_cleaned.csv  /tmp/mrjoin/ejercicio103
! hdfs dfs -put /media/notebooks/tp1/ejercicio103/clients.csv  /tmp/mrjoin/ejercicio103

mkdir: `/tmp/mrjoin/ejercicio103': File exists
put: `/tmp/mrjoin/ejercicio103/countries_cleaned.csv': File exists
put: `/tmp/mrjoin/ejercicio103/clients.csv': File exists


In [8]:
! hdfs dfs -rm /tmp/carpeta/mrjob-ejercicio103/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-ejercicio103

Deleted /tmp/carpeta/mrjob-ejercicio103/_SUCCESS
Deleted /tmp/carpeta/mrjob-ejercicio103/part-00000


In [9]:
! python mrjob-ejercicio103.py hdfs:///tmp/mrjoin/ejercicio103/* -r hadoop --python-bin /opt/anaconda/bin/python3.7 \
--output-dir hdfs:///tmp/carpeta/mrjob-ejercicio103

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio103.root.20201030.165713.088786
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio103.root.20201030.165713.088786/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio103.root.20201030.165713.088786/files/
Running step 1 of 2...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob2395509284175184545.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.4:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.4:8032

In [10]:
! hdfs dfs -tail /tmp/carpeta/mrjob-ejercicio103/part-00000

3	"Spain"
3	"Guam"
